In [31]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
dataset = load_dataset('trec')
#display(dataset)
dataset = dataset.remove_columns("label-fine")
#display(dataset)
dataset = dataset.rename_column("label-coarse", "label")
display(dataset)


train_validation_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
train_ds = Dataset.from_dict(train_validation_ds[0:4500])
validation_ds = Dataset.from_dict(train_validation_ds[4500:5000])
test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(500))])




display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Using custom data configuration default
Reusing dataset trec (C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.41it/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-bc761497303cb1ab.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-d432b7eeb4042bcd.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

,label,text
0,0,How can I transport files from one computer to...
1,3,Who are the two sons of Ozzie and Harriet Nels...
2,2,What does pH stand for ?
3,4,How many people does Honda employ in the U.S. ?
4,1,What newspaper returned a Pulitzer Prize for t...
...,...,...
4495,1,What does Final Four refer to in the sports wo...
4496,3,What is Nicholas Cage 's occupation ?
4497,1,"In order from the top , the four stripes on a ..."
4498,3,What kind of business is 7-Eleven ?


In [32]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [33]:
#creating synonyms list using wordnet
def synonyms_list_creation(target_word):
    synonyms_list = []
    try:
        for syn in wordnet.synsets(target_word):
            for lm in syn.lemmas():
                synonyms_list.append(lm.name())
        #print (set(synonyms_list))
        synonyms_list = list(set(synonyms_list))
        #transform _ to space
        synonyms_list = [s.replace('_', ' ') for s in synonyms_list]


        synonyms_list
        synonyms_list = [s for s in synonyms_list if ' ' not in s]
    
    except Exception as e:
        pass
    
    return synonyms_list

In [34]:
#senticnet part
senticnet_df = pd.read_excel(r'C:\Users\DELL\Text_Augmentation\senticnet\senticnet.xlsx',header=0)

In [35]:
#transform _ to space

senticnet_df['CONCEPT'] = senticnet_df['CONCEPT'].str.replace('_', ' ')


senticnet_df.set_index('CONCEPT',inplace=True)

print(senticnet_df)

               INTROSPECTION  TEMPER  ATTITUDE  SENSITIVITY PRIMARY EMOTION  \
CONCEPT                                                                       
abandon               -0.165     0.0     0.000        0.000     #melancholy   
abase                  0.000     0.0    -0.890        0.000       #loathing   
abash                  0.000     0.0    -0.341       -0.442           #fear   
abashment              0.000     0.0    -0.329        0.000        #dislike   
abate                  0.000     0.0    -0.339       -0.551           #fear   
...                      ...     ...       ...          ...             ...   
zotob worm             0.000     0.0     0.000       -0.958         #terror   
zotob wrestle          0.000     0.0     0.000       -0.962         #terror   
zotob wriggle          0.000     0.0     0.000       -0.918         #terror   
zotob writhe           0.000     0.0     0.000       -0.979         #terror   
zz                     0.000     0.0    -0.182      

In [36]:
#creating similar words list using senticnet
def similar_polarity_words_list_creation(target_word):
    if target_word in senticnet_df.index:
        INTROSPECTION = senticnet_df.loc[target_word, 'INTROSPECTION']
        TEMPER = senticnet_df.loc[target_word, 'TEMPER']
        ATTITUDE = senticnet_df.loc[target_word, 'ATTITUDE']
        SENSITIVITY = senticnet_df.loc[target_word, 'SENSITIVITY']





        extracted_df = senticnet_df[(senticnet_df['INTROSPECTION'] == INTROSPECTION) & 
                                   (senticnet_df['TEMPER'] == TEMPER) & (senticnet_df['ATTITUDE'] == ATTITUDE) & 
                                    (senticnet_df['SENSITIVITY'] == SENSITIVITY)]


        similar_polarity_words_list = extracted_df.index.values.tolist()
        similar_polarity_words_list = [s for s in similar_polarity_words_list if ' ' not in s]
    
    else:
        similar_polarity_words_list = []
    
    return similar_polarity_words_list

In [37]:
import itertools
from textattack.transformations import WordSwap
class Cognate_WordSwap(WordSwap):
    
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        synonyms_list = synonyms_list_creation(word)
        cognate_list = []
        try:
            cognate_list.append(similar_polarity_words_list_creation(word))
        except Exception as e:
            pass
        
        cognate_list.append(synonyms_list)
        cognate_list = list(itertools.chain.from_iterable(cognate_list))
        cognate_list = list(set(cognate_list))

        return cognate_list


In [38]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore
from textattack.augmentation import Augmenter


transformation = Cognate_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(),
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=False, compare_against_original=True, language_nltk='eng', language_stanza='en'),
                BERTScore(min_bert_score = 0.9, model_name='bert-base-uncased', num_layers=None, score_type='f1', compare_against_original=True)]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

In [ ]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):

    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)

time_end = time.perf_counter()
tim = time_end - time_sta

In [ ]:
print(tim/3600)




In [ ]:
display(results_list)

In [ ]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


In [ ]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


In [ ]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

In [ ]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\augumented_dataset\Cognate\with_pos\allow_verb_noun_swap=False_bert_score=0.9\run3\dataset.csv')